# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
cities = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities.dropna()
cities.head()

,city_id,City,Country,City ID,Cloudiness,Humidity,Longitude,Latitude,Max Temp,Wind Speed
0,0,farah,AF,1142263,0,23,62.50,32.50,74.05,2.37
1,1,tagab,AF,1123424,40,52,69.65,34.85,75.20,0.51
2,2,yellowknife,CA,6185377,90,88,-114.35,62.46,64.40,3.36
3,3,namibe,AO,3347019,1,89,12.15,-15.20,62.01,3.18
4,4,ushuaia,AR,3833367,75,74,-68.30,-54.80,37.40,4.52


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [12]:
# Store 'Lat' and 'Lng' into  locations
locations = cities[["Latitude", "Longitude"]]
humidity = cities["Humidity"].astype(float)

In [13]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer=gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
temp_cities=cities.loc[(cities['Max Temp']>=70)&(cities['Max Temp']<=80)]
wind_cities=temp_cities.loc[(temp_cities['Wind Speed']<=10)]
ideal_weather=wind_cities.loc[(wind_cities['Cloudiness']==0)]
ideal_weather.head()

,city_id,City,Country,City ID,Cloudiness,Humidity,Longitude,Latitude,Max Temp,Wind Speed
0,0,farah,AF,1142263,0,23,62.50,32.50,74.05,2.37
54,54,marsa matruh,EG,352733,0,88,27.25,31.35,75.20,5.82
154,154,codo,BR,3402000,0,60,-43.89,-4.46,72.75,1.50
196,196,puerto quijarro,BO,3444201,0,35,-57.77,-17.78,70.47,2.13
220,220,sakib,JO,247068,0,50,35.81,32.28,75.20,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:

hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAA8456XvmfzTLm28usCfjsmgm390TE_wufYIFg0Q6M5bfixqaIJ8UaUwJEj_ncetZmDpegfm6kHGWPzvLJirdl2xS4HcYiDWCekeyLh1IyRGEQzqTHjKI_T1GmhqKVVW6C7ysiQHJkOCLgsSy80LMx8N_y-JCohM5Yu3C-SycZCKQIPEnei_1ZqNhbhjM01WImsE8e4amlf0_y8UpBIIkQb7c8KRTHjWWJIAF3UyMZF4xr_Wc56dAJ7FF2tVI6V_osb7offCqsHNI_UPLvQI2tKxMtUUO_ov0zcBdhLekca1lRDRm7MgY1nQx3nanuCCl3aKgrLfakHCMSwTTS0Bu5tijwSYZPo0zU8M47eSI6AcPZQyjZBnFSFcM4Jcau6QKSL86-aaSDS-FhZGd7Td2d4SEJXvRWl-gPJlAtRq1PiQZu4aFDXegXraio_UcX0QCVyJEYFcj_CN",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 31.36208969999999,
                    "lng": 27.2218607
                },
                "viewport": {
                    "northeast": {
                        "lat": 31.36331342989272,
            

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -4.4723277,
                    "lng": -43.8914451
                },
                "viewport": {
                    "northeast": {
                        "lat": -4.471236920107279,
                        "lng": -43.88963337010728
                    },
                    "southwest": {
                        "lat": -4.473936579892722,
                        "lng": -43.89233302989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f818b80606fd1f78220f828ffe0a4e727a90ec19",
            "name": "FC Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1348,
                    "htm

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -17.80257,
                    "lng": -57.77616099999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -17.791946,
                        "lng": -57.7601536
                    },
                    "southwest": {
                        "lat": -17.8131934,
                        "lng": -57.79216839999999
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png",
            "id": "3c0365e963abed1e2dbbe01f62b2be3d80459737",
            "name": "Puerto Quijarro",
            "photos": [
                {
                    "height": 1392,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/105926428912624592482\">A Google User</a>"


{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -5.5367539,
                    "lng": -40.77358359999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -5.535394520107277,
                        "lng": -40.77223482010726
                    },
                    "southwest": {
                        "lat": -5.538094179892721,
                        "lng": -40.77493447989271
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "Golden Palace Hotel",
            "photos": [
                {
                    "height": 581,
                    "html_attribu

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAAD0NUfPmLz0jDmzekPXZfQyFla2eihhKxw8x_fA-d7KL5tl8RE4RIWiMLdzQbwnfdzAbp8y-0ZS4z0ahtzIKiaDNtYyUQLxXpVoV5GWh5UcbnD5UwUsKefKXm1wC5YbsdG8z17EoBnsm1WrCoEUJGnpkrkoj4MI3BpqkS1W79KRUnb5N8AYmmXHBNrC8GzIS0ggIujENlikBIhPR503jzm0noXf_H0wG_A8cCi9rR3FmB7PuOnqbJs6CoI_Pgj-OzRYo-1ZdemHF_iZfFz7mVXTnhIhKAm6_PU1XxocFbps_jeBAbx6ljlhTu4UlWiquNPyCcBtgKQgzHpsvhqzI0zwv71KOTPYtGoyLnih-awRY_upVSnfqutlLU8Tr-zXwZB75VCjYtVQgYGgEjyyo3rUSEA81YOH66AVZf2cmkR4iWIgaFF_Lkl89bI0HZR--XykZbGicNydC",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 39.2135907,
                    "lng": 9.12186
                },
                "viewport": {
                    "northeast": {
                        "lat": 39.21493222989272,
                        "lng": 9.123249129892722
                    },
                    "southwest": {
                        "lat":

...

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -6.0250906,
                    "lng": -44.2492866
                },
                "viewport": {
                    "northeast": {
                        "lat": -6.023839070107277,
                        "lng": -44.24800887010728
                    },
                    "southwest": {
                        "lat": -6.026538729892721,
                        "lng": -44.25070852989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f7f2cd0f348cb06f104a3e702821e20c8f3659de",
            "name": "Pousada e Restaurante Alvorada",
            "photos": [
                {
                    "height": 2560,
                    "html_attributions": [
                        "<a href=\"htt

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 45.4172825,
                    "lng": 40.5515403
                },
                "viewport": {
                    "northeast": {
                        "lat": 45.41862222989272,
                        "lng": 40.55260247989271
                    },
                    "southwest": {
                        "lat": 45.41592257010728,
                        "lng": 40.54990282010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "Luxury House ECO-hotel & Spa",
            "photos": [
                {
                    "height": 2448,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/116162205921493475602\">A Google Us

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -9.8756884,
                    "lng": -56.0885962
                },
                "viewport": {
                    "northeast": {
                        "lat": -9.874199870107278,
                        "lng": -56.08720402010727
                    },
                    "southwest": {
                        "lat": -9.876899529892722,
                        "lng": -56.08990367989271
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "0fe070edcdb7b8f4c62f5a1f66cbefedb1ecbf00",
            "name": "Lisboa Palace Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 600,
              

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 43.3374375,
                    "lng": 52.8571812
                },
                "viewport": {
                    "northeast": {
                        "lat": 43.33878067989273,
                        "lng": 52.85860127989272
                    },
                    "southwest": {
                        "lat": 43.33608102010729,
                        "lng": 52.85590162010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "18eeca56861558367cebf686cd1a4de9656a5162",
            "name": "Hotel BasTau",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1365,
                    "html

,city_id,City,Country,City ID,Cloudiness,Humidity,Longitude,Latitude,Max Temp,Wind Speed,Hotel Name
0,0,farah,AF,1142263,0,23,62.50,32.50,74.05,2.37,
1,54,marsa matruh,EG,352733,0,88,27.25,31.35,75.20,5.82,Belle Vue Hotel Marsa Matrouh
2,154,codo,BR,3402000,0,60,-43.89,-4.46,72.75,1.50,FC Hotel
3,196,puerto quijarro,BO,3444201,0,35,-57.77,-17.78,70.47,2.13,Puerto Quijarro
4,220,sakib,JO,247068,0,50,35.81,32.28,75.20,3.36,The Olive Branch Hotel
5,248,pontes e lacerda,BR,3453060,0,34,-59.34,-15.23,72.27,4.27,Plaza Hotel
6,297,waddan,LY,2209055,0,69,16.14,29.16,73.45,4.36,
7,304,novo oriente,BR,3393783,0,55,-40.77,-5.53,72.57,6.17,Golden Palace Hotel
8,380,coria,ES,2519234,0,55,-6.54,39.99,77.00,1.99,Hotel Ahc Palacio Coria
9,383,cagliari,IT,2525471,0,83,9.09,39.24,73.99,5.82,Palazzo Doglio


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))

In [30]:
hotel_df.head(20)

,city_id,City,Country,City ID,Cloudiness,Humidity,Longitude,Latitude,Max Temp,Wind Speed,Hotel Name
0,0,farah,AF,1142263,0,23,62.50,32.50,74.05,2.37,
1,54,marsa matruh,EG,352733,0,88,27.25,31.35,75.20,5.82,Belle Vue Hotel Marsa Matrouh
2,154,codo,BR,3402000,0,60,-43.89,-4.46,72.75,1.50,FC Hotel
3,196,puerto quijarro,BO,3444201,0,35,-57.77,-17.78,70.47,2.13,Puerto Quijarro
4,220,sakib,JO,247068,0,50,35.81,32.28,75.20,3.36,The Olive Branch Hotel
5,248,pontes e lacerda,BR,3453060,0,34,-59.34,-15.23,72.27,4.27,Plaza Hotel
6,297,waddan,LY,2209055,0,69,16.14,29.16,73.45,4.36,
7,304,novo oriente,BR,3393783,0,55,-40.77,-5.53,72.57,6.17,Golden Palace Hotel
8,380,coria,ES,2519234,0,55,-6.54,39.99,77.00,1.99,Hotel Ahc Palacio Coria
9,383,cagliari,IT,2525471,0,83,9.09,39.24,73.99,5.82,Palazzo Doglio
